In [80]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from scipy.special import softmax

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix

import os


In [81]:
basepath_1 = 'C:\Academic\ECE - 4370 (Machine_Learning)\Assignments\Assignment_4\Celegans_ModelGen/1'
labelled_images = []
for image in os.listdir(basepath_1):
    if os.path.isfile(os.path.join(basepath_1, image)):
        img = mpimg.imread(os.path.join(basepath_1, image))
        # remove line below to insert 101x101 images
        arr_img = img.reshape([10201])
        arr_img = np.hstack((arr_img, 1))
        labelled_images.append(arr_img)
        
basepath_0 = 'C:\Academic\ECE - 4370 (Machine_Learning)\Assignments\Assignment_4\Celegans_ModelGen/0'
for image in os.listdir(basepath_0):
    if os.path.isfile(os.path.join(basepath_0, image)):
        img = mpimg.imread(os.path.join(basepath_0, image))
        arr_img = img.reshape([10201])
        arr_img = np.hstack((arr_img, 0))
        labelled_images.append(arr_img)
np.random.shuffle(labelled_images)
# W is (M+1)xK -> 2x2
W = np.random.randn(10201+1,2)

In [56]:
m = W.shape[0] - 1
k = W.shape[1]
t = np.zeros((1, k))
for image in labelled_images:
    target = np.zeros(k)
    label = image[-1]
    target[int(label)] = 1
    t = np.vstack((t, target))
t = np.delete(t, 0, axis=0)

In [57]:
x = []
for image in labelled_images:
    image = np.delete(image, -1, axis=0)
    x.append(image)


In [58]:
x = np.array(x).reshape(len(x), 10201)
x = np.hstack((x, np.ones((len(x), 1))))

In [86]:
train_size = 9000

x_train = x[:train_size,:]
x_test = x[train_size:,:]
x_train.shape
t_train = t[:train_size,:]
t_test = t[train_size:,:]

In [88]:
phi = 0.0001
max_iter = 10000
k = W.shape[1] # num classes
for _ in range(max_iter):
    # error = -(1 / m) * (np.dot(t.T, np.log(y)) + np.dot((1 - t).T, np.log(1 - y)))
    a = np.dot(x_train, W)
    y_train = softmax(a, axis=1)
    for j in range(k):
        sum_error_gradient = 0
        sum_error_gradient = np.dot((y_train[:,j] - t_train[:,j]), x_train)
        if sum_error_gradient.all() < 10e-6:
            break
        error = sum_error_gradient.reshape(10202,1)
        W[:, j:j+1] = W[:, j:j+1] - (phi * error)

In [89]:
y_train[:,0].shape

NameError: name 'y_train' is not defined

In [72]:
W

array([[  6.94708697,  -4.86543632],
       [  5.19622449,  -6.87663212],
       [  3.86302956,  -4.375083  ],
       ...,
       [ -3.05194943,   3.91039999],
       [ -2.58991169,   3.84712369],
       [ 11.24406037, -10.78290008]])

In [73]:
""" df_w = pd.DataFrame(W)
df_w.to_csv("weights.csv") """

In [90]:
y_test = softmax(np.dot(x_test, W), axis=1)

In [91]:
y_pred = np.array([9,9])
for preds in y_test:
    index = np.argmax(preds)
    arr = np.zeros(len(preds))
    arr[index] = 1
    y_pred = np.vstack((y_pred, arr))
y_pred = np.delete(y_pred, 0, axis=0)

In [92]:
# this should be very close to t
y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [93]:
cf_matrix = multilabel_confusion_matrix(t_test, y_pred, labels=[False, True])
print(cf_matrix)

[[[192 794]
  [103 911]]

 [[911 103]
  [794 192]]]


In [98]:
correct = 0
for i, preds in enumerate(y_pred):
    if np.array_equal(preds, t_test[i]):
        correct += 1

In [99]:
correct / 2000
       

0.5515

In [70]:
# X is Nx(M+1) -> Nx2 
# used to test logistic regression implementation
""" df = pd.read_csv("C:\Developer\ML_PR\ML\logistic_regression\classification.csv")
x = df.iloc[:,:-1]
x = np.hstack((x, np.ones((x.shape[0],1))))
# fix t to make it one-hot encoded
classes = df.loc[:,'success'].astype('int64') """

""" 
for test in classes:
    if test:
        t = np.vstack((t, [1, 0]))
    else:
        t = np.vstack((t, [0, 1]))
t = np.delete(t, 0, axis=0) """

' \nfor test in classes:\n    if test:\n        t = np.vstack((t, [1, 0]))\n    else:\n        t = np.vstack((t, [0, 1]))\nt = np.delete(t, 0, axis=0) '